In [138]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [139]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [140]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [141]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError



response = table.scan()
alldata = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    alldata.extend(response['Items'])

In [146]:
len(alldata)

168870

In [147]:
for index_, row_ in enumerate(alldata):
  row_['courseid'] = row_['metadata'][:6]


In [148]:
with table.batch_writer(overwrite_by_pkeys=['metadata', 'id']) as batch:
  for index_, row_ in enumerate(alldata):
    print(index_, row_['id'], end='\r')
    batch.put_item(
      Item= row_
    )

In [149]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

courseid = "MGC004"
date = "20230919"


response = table.scan(
    # IndexName='id-courseid-index',
    FilterExpression=Attr('metadata').begins_with(courseid)&Attr('date').eq(date)
)

data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(
      # IndexName='id-courseid-index',
      FilterExpression=Attr('metadata').begins_with(courseid)&Attr('date').eq(date),
      ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [150]:
len(data)

8635

In [153]:
courseid = "MGC004"
date = "20230919"


response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  FilterExpression=Attr('date').eq(date))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  ExclusiveStartKey=response['LastEvaluatedKey'] ,FilterExpression=Attr('date').eq(date) )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [154]:
len(data)

8635